This notebook makes use of https://github.com/akabe/ocaml-jupyter with ocaml 4.02.3

# Prime fields
Let's use our module to build Frobenius fields, aka prime fields $ \mathbf{F}_p $.

First, we need to load the functor that makes a field based on a characteristic `carac`, which must be a prime.

In [9]:
#mod_use "../src/frobenius.ml"

module Frobenius :
  sig
    module type FieldT =
      sig
        type t
        val zero : t
        val un : t
        val alpha : t
        val carac : int
        val dim : int
        val cardinal : int
        val to_int : t -> int
        val of_int : int -> t
        val to_list : t -> int list
        val elt_of_list_int : int list -> t
        val print : t -> unit
        val print_table : unit -> unit
        val add : t -> t -> t
        val sub : t -> t -> t
        val opp : t -> t
        val mult : t -> t -> t
        val inv : t -> t
        val div : t -> t -> t
        val pow : t -> int -> t
        val random : unit -> t
      end
    module Frobenius : functor (Taille : sig val carac : int end) -> FieldT
  end


Now, let's build $ \mathbf{F}_3  = \mathbf{Z}/(3\mathbf{Z})$ :

In [10]:
module T3 = struct
    let carac = 3
end

module F3 = Frobenius.Frobenius(T3)

module T3 : sig val carac : int end


module F3 :
  sig
    type t = Frobenius.Frobenius(T3).t
    val zero : t
    val un : t
    val alpha : t
    val carac : int
    val dim : int
    val cardinal : int
    val to_int : t -> int
    val of_int : int -> t
    val to_list : t -> int list
    val elt_of_list_int : int list -> t
    val print : t -> unit
    val print_table : unit -> unit
    val add : t -> t -> t
    val sub : t -> t -> t
    val opp : t -> t
    val mult : t -> t -> t
    val inv : t -> t
    val div : t -> t -> t
    val pow : t -> int -> t
    val random : unit -> t
  end


In [11]:
F3.(print alpha)

- : unit = ()


Unfortunately, we cannot print in here directly. We'll just need to flush standard output, which can be done easily  done `print_newline()` or `print_endline()` cf https://caml.inria.fr/pub/docs/manual-ocaml/libref/Pervasives.html :

In [12]:
print_newline ()

2%3


- : unit = ()


We can now see that the 2 is a (the) primitive element in this field. 
If we need more flexible representation, we can just use other primitives :

In [13]:
F3.(to_int alpha) |> print_int ; print_newline ()

2


- : unit = ()


Let's extend this field to make $ \mathbf{F}_9 $. To do this, we're actually going to build $ \mathbf{F}_3[X]/(f(x))$ where $f(x)$ is an irreducible polynomial of degree 2.
First, we need to make polynoms out of these. Then, the module Cantorzass factorizes a cyclotomic polynomial and exten

In [14]:
#mod_use "../src/polynome.ml"
#mod_use "../src/cantorzass.ml"
#mod_use "../src/cyclo.ml"
#use "../src/extensions.ml"

module Polynome :
  sig
    module type CorpsType = Frobenius.FieldT
    module type PolyType =
      sig
        type t
        type elt
        val carac : int
        val cardinal : int
        val of_list : elt list -> t
        val to_list : t -> elt list
        val nul : t
        val un : t
        val coef_dom : t -> elt
        val map : (elt -> elt) -> t -> t
        val evaluer : t -> elt -> elt
        val normalise : t -> t
        val unitaire : t -> t
        val decale : t -> int -> t
        val monome : elt -> int -> t
        val reciproque : t -> t
        val coef_constant : t -> elt
        val degre : t -> int
        val derive : t -> t
        val opp : t -> t
        val add : t -> t -> t
        val sub : t -> t -> t
        val mult : t -> t -> t
        val pow : t -> int -> t
        val powmod : t -> int -> t -> t
        val division : t -> t -> t * t
        val quotient : t -> t -> t
        val modulo : t -> t -> t
        val mult_par_scal : t -> el

      val random : unit -> t
    end
module ExtensionOpt :
  functor (CorpsBase : CFT) (Taille : TT) ->
    sig
      type t
      type elt = CorpsBase.t
      exception ExtraireImpossible
      module Poly :
        sig
          type t
          type elt = elt
          val carac : int
          val cardinal : int
          val of_list : elt list -> t
          val to_list : t -> elt list
          val nul : t
          val un : t
          val coef_dom : t -> elt
          val map : (elt -> elt) -> t -> t
          val evaluer : t -> elt -> elt
          val normalise : t -> t
          val unitaire : t -> t
          val decale : t -> int -> t
          val monome : elt -> int -> t
          val reciproque : t -> t
          val coef_constant : t -> elt
          val degre : t -> int
          val derive : t -> t
          val opp : t -> t
          val add : t -> t -> t
          val sub : t -> t -> t
          val mult : t -> t -> t
          val pow : t -> int -> t
          val

In [15]:
module T6 = struct
    let carac = 3
    let dim = 2
end

module F9 = ExtensionOpt(F3)(T6)

module T6 : sig val carac : int val dim : int end


module F9 :
  sig
    type t = ExtensionOpt(F3)(T6).t
    type elt = F3.t
    exception ExtraireImpossible
    module Poly :
      sig
        type t = ExtensionOpt(F3)(T6).Poly.t
        type elt = elt
        val carac : int
        val cardinal : int
        val of_list : elt list -> t
        val to_list : t -> elt list
        val nul : t
        val un : t
        val coef_dom : t -> elt
        val map : (elt -> elt) -> t -> t
        val evaluer : t -> elt -> elt
        val normalise : t -> t
        val unitaire : t -> t
        val decale : t -> int -> t
        val monome : elt -> int -> t
        val reciproque : t -> t
        val coef_constant : t -> elt
        val degre : t -> int
        val derive : t -> t
        val opp : t -> t
        val add : t -> t -> t
        val sub : t -> t -> t
        val mult : t -> t -> t
        val pow : t -> int -> t
        val powmod : t -> int -> t -> t
        val division : t -> t -> t * t
        val quotient : t -> t -> t
   

In [16]:
let _ = 
    let open F9 in
    let rec enum x = match x with
        | a when a=un -> print un ; print_newline() ;
        | a -> print a ; print_newline() ; enum (mult a alpha)
    in enum alpha
;
print_newline() ; F9.print_table ()

a^1
a^2
a^3
a^4
a^5
a^6
a^7
a^0

1%3X^0

0%3X^0 + 1%3X^1

1%3X^0 + 2%3X^1

2%3X^0 + 2%3X^1

2%3X^0

0%3X^0 + 2%3X^1

2%3X^0 + 1%3X^1

1%3X^0 + 1%3X^1



- : unit = ()


In [19]:
F9.(Poly.decale un 1)

error: compile_error

In [20]:
module F9' = ExtensionNonOpt(F3)(T6)

module F9' :
  sig
    type t = ExtensionNonOpt(F3)(T6).t
    type elt = F3.t
    exception ExtraireImpossible
    module Poly :
      sig
        type t = ExtensionNonOpt(F3)(T6).Poly.t
        type elt = elt
        val carac : int
        val cardinal : int
        val of_list : elt list -> t
        val to_list : t -> elt list
        val nul : t
        val un : t
        val coef_dom : t -> elt
        val map : (elt -> elt) -> t -> t
        val evaluer : t -> elt -> elt
        val normalise : t -> t
        val unitaire : t -> t
        val decale : t -> int -> t
        val monome : elt -> int -> t
        val reciproque : t -> t
        val coef_constant : t -> elt
        val degre : t -> int
        val derive : t -> t
        val opp : t -> t
        val add : t -> t -> t
        val sub : t -> t -> t
        val mult : t -> t -> t
        val pow : t -> int -> t
        val powmod : t -> int -> t -> t
        val division : t -> t -> t * t
        val quotient : t -> t -

In [23]:
F9'.(print alpha)

0%3X^0 + 1%3X^1


- : unit = ()


In [26]:
F9.(print alpha) ; print_newline ()

a^1a^1


- : unit = ()
